In [ ]:
%pylab inline
import torch
from torch.autograd import Variable
from torch import nn
import loader
from threes import *
from qLearningNet import *
import time
from AIModels import *


MOVES = [0, 1, 2, 3]
CUDA = False

FILENAME = "saved_parameters"
INPUT_SIZE = 19
HIDDEN_SIZE = 256
GAMMA = 0.8

def testNet(filename=None, net=None):
    game = Threes(save_game=False)
    if filename is not None:
        ai = QLearningNetAI(game, filename=filename)
    elif net is not None:
        ai = QLearningNetAI(game, net=net)
    scores, move_count, highs = AIModel.test_ai(ai, 100, verbose=False)
    return move_count, np.mean(scores), np.mean(highs)


# parametry sieci np. learning rate
# lepszy zbior uczacy - jak w DeepMindzie
# nagrody - musza byc wieksze - inaczej sami sobie redukujemy learning rate
# (gestsze nagrody - np. logarytm z rewardow)
# uczyc jednak dobrymi przebiegami
# http://pytorch.org/docs/master/nn.html#embedding 

In [ ]:
def train(model, data_loaders, optimizer, num_epochs=500, log_every=100, verbose=True):
    if CUDA:
        model.network.cuda()
        
    results = []
    losses = []
    data = np.zeros((1000,40))
    data_ptr = 0
    
    epoch = 0
    if verbose:
        print u'Training the model!'
        print u'Interrupt at any time to get current model'
    try:
        while epoch < num_epochs:
            epoch += 1
            x = data_loaders.get(model, 30)
            #data[data_ptr:data_ptr + 30] = x
            #data_ptr += 30
            #data_ptr %= 1000 - 30
            
            #random_data_ptr = np.random.randint(0,data_ptr)
            #x = data[random_data_ptr : random_data_ptr + 30]
            
            future = x[:, 21:]
            future_scores = model.Q(future)
            for i, row in enumerate(future):
                #print(row)
                game = Threes(save_game=False, data=row.tolist())
                for j, move in enumerate(MoveEnum):
                    if not game.canMove(move):
                        future_scores[i, j] = float('-inf')
                if not game.getPossibleMoves():
                    future_scores[i:,:] = np.full((1,4), x[i, 20])
                
            y = x[:, 20] + GAMMA * np.max(future_scores, axis=1)
            xx = x[:, :19]

            optimizer.zero_grad()
            i = np.asarray(np.vstack((np.arange(0,30),x[:,19])),int)
            
            i = torch.LongTensor(i)
            out = model.Q(xx, as_variable=True)[i[0],i[1]]
            loss = model.loss(out, y)
            loss.backward()
            optimizer.step()

            if epoch % log_every == 0 and verbose:
                print u"Minibatch {0: >6}  | loss {1: >15.12f} ".format(epoch, loss.data[0])
                result = testNet(net=model)
                print u"The average score was {}".format(result[1])
                results += [result[1]]
                losses += [loss.data.numpy()]
            if epoch % (log_every * 20) == 0:
                result = testNet(net=model)
                filename = FILENAME +'{}_{}'.format(epoch, int(result[1]))
                model.save_parameters(filename)
                

    except KeyboardInterrupt:
        pass
    result = testNet(net=model)
    filename = FILENAME +'{}_{}'.format(epoch, int(result[1]))
    model.save_parameters(filename)
    return results, losses

In [ ]:
q_learning_net = QLearningNet()
for p in q_learning_net.network.parameters():
    p.requires_grad = True
optimizer = torch.optim.Adam(q_learning_net.network.parameters(), lr=100.000)
data_loader = loader.Loader()
scores, loses = train(q_learning_net, data_loader, optimizer, num_epochs=20000)
print("Learning done")
# save scores, loses to file?

In [ ]:
figure()
figsize(16, 4)
subplot(1,2,1)
plot(range(100,len(scores)*100+1, 100), scores, marker='o')
xlabel('Number of epochs')
ylabel('Average score')
subplot(1,2,2)
plot(range(100,len(scores)*100+1, 100), np.log(loses), marker='o')
xlabel('Number of epochs')
ylabel('Loss')
show()

In [ ]:
filename = FILENAME + '6000_447'
print testNet(filename = filename)

In [ ]:
game = Threes(save_game=False)
ai = RandomAI(game)
scores, move_count, highs = AIModel.test_ai(ai, 1000, verbose=False)
print move_count
print np.mean(scores), np.mean(highs)

In [ ]:
game = Threes(save_game=False)
ai = MiniMaxAI(game)
scores, move_count, highs = AIModel.test_ai(ai, 10, verbose=False)
print move_count
print np.mean(scores), np.mean(highs)